In [1]:
from azureml.core import Workspace, Experiment

experiment_name = 'udacity-project-local'

subscription = '2c48c51c-bd47-40d4-abbe-fb8eabd19c8c'
rg = 'aml-quickstarts-190128'
workspace = 'quick-starts-ws-190128'

ws = Workspace.get(name=workspace, subscription_id=subscription, resource_group=rg)
#ws = Workspace.from_config()
exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (msrest 0.6.19 (c:\users\aldam\anaconda3\lib\site-packages), Requirement.parse('msrest>=0.6.21'), {'azure-mgmt-resource', 'azure-mgmt-keyvault', 'azure-mgmt-storage', 'azure-mgmt-containerregistry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (msrest 0.6.19 (c:\users\aldam\anaconda3\lib\site-packages), Requirement.parse('msrest>=0.6.21'), {'azure-mgmt-resource', 'azure-mgmt-keyvault', 'azure-mgmt-storage', 'azure-mgmt-containerregistry'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (msrest 0.6.19 (c:\users\aldam\anaconda3\lib\site-packages), Requirement.parse('msrest>=0.6.21'), {'azure-mgmt-resource', 'azure-mg

UserErrorException: UserErrorException:
	Message: You are currently logged-in to 660b3398-b80e-49d2-bc5b-ac1dc93b5254 tenant. You don't have access to 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Udacity CloudLabs Sub - 09', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f'), SubscriptionInfo(subscription_name='Udacity CloudLabs Sub - 07', subscription_id='48a74bb7-9950-4cc1-9caa-5d50f995cc55'), SubscriptionInfo(subscription_name='Udacity CloudLabs Sub - 06', subscription_id='5a4ab2ba-6c51-4805-8155-58759ad589d8'), SubscriptionInfo(subscription_name='Udacity CloudLabs Sub - 04', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": 

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name= 'cpu-compute-trn'
compute_size= 'Standard_D2_V2'

try:
    cpu_cluster_compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=compute_size, max_nodes=4)
    cpu_cluster_compute_target = ComputeTarget.create(ws, compute_name, compute_config)
cpu_cluster_compute_target.wait_for_completion(show_output=True)

print(cpu_cluster_compute_target.get_status().serialize())

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice
from azureml.core import ScriptRunConfig, Environment
import os

# Specify parameter sampler
#ps  = ### YOUR CODE HERE ###
ps=RandomParameterSampling({
    "C": choice(100, 10, 1.0, 0.1, 0.01),
    "max_iter": choice(range(50,200))}
)

# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1, 
                      evaluation_interval=1, 
                      delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
est = SKLearn(source_directory='.',
            entry_script='train.py',
            compute_target=cpu_cluster_compute_target)

#est = ScriptRunConfig(source_directory=".",
#                     script="train.py",
#                     compute_target=cpu_cluster_compute_target,
#                     environment=env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
#parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
#print('\n Accuracy:', best_run_metrics['accuracy'])
#print('\n C:',parameter_values[0])
#print('\n max_iter:',parameter_values[1])

hyperdrive_run.wait_for_completion(show_output=True)

os.makedirs("./outputs", exist_ok=True)
joblib.dump( value=best_run, filename='outputs/best_run_model.pkl')
print('Model has been sucessfully saved!')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

ds = ds.register(workspace=ws, 
                name='bankmarketing',
                description='Project 1 dataset',
                tags={'format':'CSV'})

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name='y',
    compute_target=cpu_cluster_compute_target,
    n_cross_validations=5)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

exp_aml = Experiment(ws, 'autoML')
run = exp_aml.submit(config=automl_config, show_output=True)

RunDetails(run).show()

run.wait_for_completion(show_output=True)

os.makedirs("./outputs", exist_ok=True)
joblib.dump( value=run, filename='outputs/automml_model.pkl')
print('Model has been sucessfully saved!')